In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import scipy.optimize
import scipy.stats
from scipy.io import loadmat
from datetime import datetime, date, time, timedelta
sns.set()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Taisa's code / adapted fxn

In [ ]:
# code to build a recurrence plot
# Taisa Kushner, Dec 4 2023


# To convert to patient data, just replace vector a with whatever vector you want to use, and you can delete all the print statements that are inside the loop as they will slow things down.

# first let's build a random vector
# we'll do something that is easy to verify the correct construction of the recurrence plot for
a = np.array([1,2,1,2,3,1])
print('We are building a recurrence plot for the following vector :')
print(a)

# let's get the length of a
N = len(a)

# build an empty identify matrix to hold our recurrenc plot matrix, as we know the diagonal all matches
recMat = np.eye(N)

# now we will take two copies of the array "a" and slide them against one another. So at step "j" we are comparing N-j elements with the ones next to them. This will give us the values that belong on the "jth" diagonal of the recurrence matrix. We will then set that j-th diagonal equal to this compared array.

# we'll do the lower triangular matrix first
for j in range(1,N):
    np.fill_diagonal(recMat[j:,:-j], np.equal(a[j:],a[:-j]))
    # let's add in some print statements so you can see what the code is doing
    print('------')
    print('We are building the off-diagonal that is ('+ str(j)+') below the main diagonal')
    print('The first element of the first array should be: '+str(a[j]))
    print('First array being compared is: ')
    print(a[j:])
    print('Second array being compared is: ')
    print(a[:-j])

# now we will transpose this matrix, and add it to itself to fill the upper triangular matrix
print('-- Our intermediate lower triangular array is: ')
print(recMat)

print('Now we are adding the transpose...')
print('The transpose is: ')
print( np.transpose(recMat))

recMat = recMat + np.transpose(recMat)
print('Our matrix now looks like:')
print(recMat)

# this will yield 2's on the main diagonal, so let's subtract the N-dimensional identiy matrix from this to get that back to 1's
print('Now we subtract the identity to get our main diagonal values back to 1')

recMat = recMat - np.eye(N)

print('---Our final matrix is---')
print(recMat)

In [2]:
def build_RQA_plot(df, var):
    a = df[var].values.flatten()
    N = len(a)
    recMat = np.eye(N)
    
    for j in range(1,N):
        np.fill_diagonal(recMat[j:,:-j], np.equal(a[j:],a[:-j]))

    recMat = recMat + np.transpose(recMat)

    recMat = recMat - np.eye(N)

    return recMat

## Coarse matrices

In [8]:
c_data = glob.glob('coarse/hashes_updated/*****_coarse.csv')


In [30]:
ids = []
for name in c_data:
    ids.append(name[22:27])

In [10]:
c_hash_dfs = []
for file in c_data:
    c_hash_dfs.append(pd.read_csv(file, low_memory = False))

In [32]:
for i in range(len(c_hash_dfs)):
    df = c_hash_dfs[i]
    c_mat = build_RQA_plot(df, 'hash')
    
    csv_name = 'coarse/results_updated/' + ids[i] + '_coarse_mat.csv'
    temp_df = pd.DataFrame(c_mat)
    temp_df.to_csv(csv_name, index = False, header = False)

## Fine matrices

In [8]:
f_data = glob.glob('fine/hashes_updated/*****_fine.csv')

In [11]:
f_ids = []
for name in f_data:
    f_ids.append(name[20:25])

In [13]:
f_hash_dfs = []
for file in f_data:
    f_hash_dfs.append(pd.read_csv(file, low_memory = False))

In [14]:
for i in range(len(f_hash_dfs)):
    df = f_hash_dfs[i]
    f_mat = build_RQA_plot(df, 'hash')
    
    csv_name = 'fine/results_updated/' + f_ids[i] + '_fine_mat.csv'
    temp_df = pd.DataFrame(f_mat)
    temp_df.to_csv(csv_name, index = False, header = False)